# LocalCart scenario part 2: Creating a streaming pipeline

<a id="intro"></a>
## Introduction


A web or mobile app will trigger events as a user navigates a web site. These clickstream events indicate when a customer logs in, adds something to a basket, completes an order, and logs out. The events are placed into configured Message Hub (Apache Kafka) that provides a scalable way to buffer the data before it is saved, analysed, and rendered. 

[Notebook #1 - Creating a Kafka Producer of ClickStream events](https://apsportal.ibm.com/analytics/notebooks/c3aee820-01af-478f-bd0f-07d80866863f/view?projectid=81238e6c-a19b-4c5c-9e45-753dfe7b7de3&context=analytics) generates clickstream events for LocalCart and sends them to Message Hub to show how data can be collected offline and streamed to the cloud later. A [Java app](https://localcartkafkaproducer.mybluemix.net/LocalCartKafkaProducer/) continuously feeds a simulated stream of events to Message Hub. 

This notebook creates a streaming pipeline that ingests those clickstream events, processes each event type, aggregates the events for real-time analysis, and streams them to Redis storage on the cloud for later analysis.

This notebook runs on Python 2 with Spark 2.0.

### Static data analysis

In order to analyse the data at rest, we can create a streaming pipline job that takes the data from Message  Hub and stores it in CSV files. These files can be concatenated and loaded into a Jupyter notebook. We can use [Pixiedust](https://github.com/ibm-cds-labs/pixiedust) to analyse the data. This type of analysis with Pixiedust is done in [Notebook #4: Visualize streaming data](https://apsportal.ibm.com/analytics/notebooks/d9fd6d78-d55f-4e83-b8ae-d465f7af256f/view?projectid=81238e6c-a19b-4c5c-9e45-753dfe7b7de3&context=analytics).


### Streaming data analysis

In order to gather aggregations of each event as it arrives, we can set up a streaming pipeline that performs aggregation operations and writes the output to Redis. Aggregation operations enable us to do some real-time analysis as the data streams.


### Visualization

After we have aggregations in Redis, we can see the data in dashboards by using Pixiedust in notebooks, or by using custom-made Node.js web applications.  

## Table of contents

1. [Introduction](#intro)<br>
2. [Scenario](#process)<br>
3. [Collect data from Message Hub](#collect)<br>
4. [Process the incoming events with Python code](#process_events)<br>
5. [Set up Aggregation functions for events](#aggregation)<br>
6. [Provision and set up Redis database](#redis)<br>
7. [Repeat for next streams ....](#repeat)<br>
8. [Summary and next steps](#summary)<br>


<a id="process"></a>
## Scenario 

In this notebook, our aim is to analyse the incoming events by using the streaming pipelines service. The following graphic shows LocalCart clickstream events that are generated and sent from the Message Hub service. 
<img src='https://github.com/notebookgraphics/advobeta/blob/master/NB2_COMPLETE_STREAMING_PIPELINE.gif?raw=true'></img>

- The first stream deals with the login event type. It is aggregated to get its count.
- The second stream deals with the add_to_cart event type. It is processed by Python code to compute the total basket price and the UTZ time stamp, and then aggregated to get the sum.
- The third stream deals with the checkout event type. It is processed by Python code to compute the total basket price and the UTZ time stamp, and then aggregated to get the sum.

We use the same Redis instance for all three streams in our pipeline.


<a id="collect"></a>
## Collect data from Message Hub

First we need to create a streaming pipeline that collects data from a Message Hub operator.


***

### Steps

In IBM Data Science Experience, do these steps:

1. Select a project that you want to contain the streaming pipeline.
1. Click the **Analytics Assets** tab
1. In the Streaming Pipelines section, click **add streaming piplelines**.
1. In the Create Streaming Pipeline window, click **Create Manually**. Type in a name for the pipeline, and then click **Continue**.
1. Drag a MessageHub source operator into the pipeline canvas.
1. Configure the MessageHub operator by doing these steps in the Properties pane:
	1. Select the ClickStream MessageHub instance.
	1. Select the topic that you want to aggregate. For example, you might select "Login".
	1. Click **Edit Schema** to match the incoming data:
        - `customer_id` - text - `.customer_id`
        - `click_event_type` - text - `.click_event_type`
        - `event_time` - text - `.event_time`


Our streaming pipeline now has its first operator and looks like this: <img src='https://github.com/notebookgraphics/advobeta/blob/master/NB2_MH.gif?raw=true'></img>

<a id="process_events"></a>
## Process the incoming events with Python code

Python code can be used to to process incoming events. The Python code has the following form:

In [47]:
def process(event):
    return {'output':'output'}

With each incoming event, the streaming pipeline will call "process" with the incoming data. Whatever your function returns will be sent to the next block. Here is an example of a login clickstream event:

In [48]:
ev = { 
    'customer_id': '14420', 
    'click_event_type': 'login',     
    'total_price_of_basket' : "0.0",
    'total_number_of_items_in_basket' : "0",
    'total_number_of_distinct_items_in_basket' : "0",
    'event_time': '2017-04-10 15:54:35 IST'
}

Let's make a "process" function that parses the time stamp and returns the parsed date:

In [51]:
from dateutil.parser import parse
from dateutil import tz
def process(event):
    # datetime.parse doesn't understant "IST" as a timezone indicator, so swap for +05:30
    dt = parse(event['event_time'].replace('IST','+05:30'))
    
    # convert to UTC timezone too
    event['dt_utc'] = dt.astimezone(tz.gettz('UTC'))

    return event

and convert our sample data with it:

In [52]:
print process(ev)

{'event_time': '2017-04-10 15:54:35 IST', 'total_number_of_items_in_basket': '0', 'dt_utc': datetime.datetime(2017, 4, 10, 10, 24, 35, tzinfo=tzfile('/usr/share/zoneinfo/UTC')), 'total_number_of_distinct_items_in_basket': '0', 'total_price_of_basket': '0.0', 'click_event_type': 'login', 'x': 'hello', 'customer_id': '14420'}


***

### Steps

In the pipeline canvas, do these steps:
1. Drag a Code operator from the Processing and Analytics area, and then drop it on the canvas next to the MessageHub operator.
2. Drag your mouse pointer from the output port of the MessageHub operator to the input port of the Code operator to connect them.
3. Click the **Code** operator to open its Properties pane. 
    - Copy and paste the following code into the Code block:

    ```
        from dateutil.parser import parse
        from dateutil import tz
        def process(event):
            # datetime.parse doesn't understant "IST" as a timezone indicator, so swap for +05:30
            dt = parse(event['event_time'].replace('IST','+05:30'))
    
            # convert to UTC timezone too
            event['dt_utc'] = dt.astimezone(tz.gettz('UTC'))

        return event
    ```

   - Click **Edit Schema** to edit the code block to match the schema in the MessageHub operator. Add a new attribute to the Code's schema: `dt_utc` which is of type Date.


You now have a streaming pipeline with two operators and that looks like this:
<img src='https://github.com/notebookgraphics/advobeta/blob/master/NB2_MH_CODE.gif?raw=true'></img>



Let's turn now to the third operator, Aggregation.

<a id="aggregation"></a>
## Set up Aggregation functions for events

Streaming data can be aggregated and then a function such as sum, count, minimum, or maximim can be done on the aggregation before it is written to the Redis database. Our aim is to calculate the following data for a sliding one-hour window:

- login_count - the number of people who logged into LocalCart 
- basket_count - the number of items added into a shopping cart
- checkout_count - the number of purchases made
- basket_total - the total price of items added into a shopping cart
- checkout_total - the total price of items purchased

Let's walk though how to assign an aggregation function for the `login_count` event type in our streaming data.

***

### Steps

In the pipeline canvas, do these steps:

- Drag a Aggregation operator from the Processing and Analytics area, and then drop it on the canvas next to the Code operator.
- Drag your mouse pointer from the output port of the Code operator to the input port of the Aggregation operator to connect them.
- Click the Aggregation operator to open its Properties pane. Set the following parameters:
    - Type - 'sliding'
    - Time Units - 'hour'
    - Number of Time Units - 1
    - Partition By - leave unchanged
    - Group By - leave unchanged
- In the Functions area of the Aggregation Properties pane, assign the following values:
    - Output Field Name - `login_count`
    - Function Type - count
    - Apply Function To - `login_count` 
     ***
     

Repeat the steps above for the `basket_count` and `checkout_count` aggregators on their respective Message Hub topics.
    
The `basket_total` and `checkout_total` aggregations are achieved by adding a second aggregation function to the existing block, this time using a "Sum" function.


Our pipeline now has three operators and looks like this:
<img src='https://github.com/notebookgraphics/advobeta/blob/master/NB2_MH_CODE_AGGREG.gif?raw=true'></img>



<a id="redis"></a>
## Provision and set up Redis database

Redis is an in-memory database. It stores its data in RAM, making it a very fast way of storing and retrieving data. It provides a set of primitive data structures, but we only concern ourselves with [hashes](https://redis.io/commands#hash) for this exercise.

A Redis hash is a data structure that allows several keys to be stored together. We are going to configure a Redis hash called "funnel" that contains the following output:

- login_count - the number of people who logged into LocalCart
- basket_count - the number of items added into a shopping cart
- checkout_count - the number of purchases made
- basket_total - the total price of items added into a shopping cart
- checkout_total - the total price of items purchased

These are the outputs of the aggregation functions in our streaming pipeline. Let's provision our own Redis service:

***

### Steps

In the IBM Bluemix Dashboard, do these steps:

1. Click the **Services** tab.
1. Choose the **Redis By Compose** service.
1. Provision a new Redis By Compose service. Note its authentication details (hostname, port, and password).

***

Now we can play with the Redis service in this notebook by installing the Python Redis library with the following command:

In [26]:
!pip install redis

We import the library with this command:

In [37]:
import redis
r = redis.StrictRedis(host='abc.com', port=10115, db=0, password='ABCDEFGHI')

We can then create a hash called 'funnel' to store our real-time data to the database by using the `hset` function:


In [1]:
r.hset('funnel', 'basket_count', 554);
r.hset('funnel', 'basket_total', 951);
r.hset('funnel', 'checkout_count', 21);
r.hset('funnel', 'checkout_total', 5400);
r.hset('funnel', 'login_count', 100);

NameError: name 'r' is not defined

We can also use this connection to retrieve all the values from our 'funnel' hash using `hgetall`:

In [1]:
r.hgetall('funnel')

NameError: name 'r' is not defined

**Note:** 
The Redis connection above seems to freeze in this notebook after a minute or so. In this case, you will need to restart the notebook kernel to restore it.


We can now store the aggregations from our streaming pipeline in Redis.

------------------------------------------------------------------------------

### Steps

In the Bluemix Dashboard, do these steps:

1. Click the **Services** tab.
1. Choose the **Redis By Compose** service.
1. Provision a new Redis By Compose service. Note its authentication details (hostname, port, and password).

Let's add a Redis output to our streaming pipelines. In the streaming pipeline canvas, do these steps:

1. Drag a Redis operator from the Target area, and then drop it on the canvas next to the Aggregator operator.
1. Drag your mouse pointer from the output port of the Aggregator operator to the input port of the Redis operator to connect them.
2. Click the **Redis** operator to open its Properties pane. 
    - Type in the value of 'hostname', 'port', and 'password' of your Redis by Compose service.
    - In the Key Template field, type in "funnel". 


***

Your pipeline should now look like this:
<img src='https://github.com/notebookgraphics/advobeta/blob/master/NB2_STREAMING_PIPELINE.gif?raw=true'></img>


Congratulations! You just created a streaming pipeline that takes clickstream data from MessageHub, processes the data with Python, aggregates the event types and applies functions to them, and then writes counts and totals to Redis storage.


<a id="repeat"></a>
## and repeat for next streams ....

You now need to create two more streams into the pipeline. One stream is for the "add_to_cart" Message Hub event type. The second stream is for the "checkout" Message Hub topic. The messages on those topics are a bit more detailed:

In [19]:
add_to_cart_event = {
    "customer_id": "13859",
    "click_event_type": "add_to_cart",
    "product_name": "Oatmeal",
    "product_category": "Food",
    "product_price": "2.49",
    "total_price_of_basket": "153.41",
    "total_number_of_items_in_basket": "19",
    "total_number_of_distinct_items_in_basket": "6",
    "event_time": "2017-06-23 12:56:18 UTC"
}
checkout_event =  {
    "customer_id": "11828",
    "click_event_type": "checkout",
    "total_price_of_basket": "72.80000000000001",
    "total_number_of_items_in_basket": "20",
    "total_number_of_distinct_items_in_basket": "5",
    "session_duration": "440",
    "event_time": "2017-06-23 13:09:12 UTC"
}

When you create those two streams, you need to add extra fields to the Message Hub schema and parse them correctly in the Python code.

In [ ]:
from dateutil.parser import parse
from dateutil import tz
def process(event):
    # datetime.parse doesn't understant "IST" as a timezone indicator, so swap for +05:30
    dt = parse(event['event_time'].replace('IST','+05:30'))
    
    # convert to UTC timezone too
    event['dt_utc'] = dt.astimezone(tz.gettz('UTC'))
    event['total_price_of_basket'] = float(event['total_price_of_basket'])
    return event



Drag a second Aggregation operator to the canvas, connect it to the Code operator for the add_to_cart event type, and define the Aggregator Properties pane with the following values:
    - Type - 'sliding'
    - Time Units - 'hour'
    - Number of Time Units - 1
    - Partition By - leave unchanged
    - Group By - leave unchanged
    - Output Field Name - `basket_total`
    - Function Type - sum
    - Apply Function To - `basket_total` 


Drag a third Aggregation operator to the canvas, connect it to the Code operator for checkout event type, and define the Aggregator Properties pane with the following values:
    - Type - 'sliding'
    - Time Units - 'hour'
    - Number of Time Units - 1
    - Partition By - leave unchanged
    - Group By - leave unchanged
    - Output Field Name - `checkout_total`
    - Function Type - sum
    - Apply Function To - `checkout_total` 


We use the same Redis instance for all three streams in our pipeline. Consequently, you only need to create the Redis By Compose service one time.

Our complete streaming pipeline now looks like this: 

<img src='https://github.com/notebookgraphics/advobeta/blob/master/NB2_COMPLETE_STREAMING_PIPELINE.gif?raw=true'></img>

<a id="summary"></a>

## Summary and next steps
In this notebook you set up a streaming pipeline that used data from notebook [Notebook #1: Creating a Kafka Producer of ClickStream events](https://apsportal.ibm.com/analytics/notebooks/c3aee820-01af-478f-bd0f-07d80866863f/view?projectid=81238e6c-a19b-4c5c-9e45-753dfe7b7de3&context=analytics). The data included clickstream events (log in, browse, add to cart, logout without purchase, logout with purchase, and checkout).  

### Author
Glynn Bird is a Developer Advocate for Watson Data Platform at IBM. 

***

Copyright © IBM Corp. 2017. This notebook and its source code are released under the terms of the MIT License.